In [19]:
import json
# from dateutil.parser import parse as parse_datetime
from datetime import datetime, timedelta
import math
from dateutil.parser import parse


def ceil_to_next_15(dt):
    dt = parse(dt)
    dt = dt.replace(second=0, microsecond=0)
    dt += timedelta(minutes=(15 - dt.minute % 15) % 15)
    return dt.strftime("%Y-%m-%dT%H:%M:%S.000Z")

# dt1 = datetime.strptime("2025-03-11 10:01 AM", "%Y-%m-%d %H:%M:%S")

# ceil_to_next_15(dt1)

In [4]:
import json
with open(r"C:\Users\Vishwas\Downloads\eda752c3-ac69-4752-bce6-f718805822bf_MEAL.json") as f:
    food_data = json.load(f)
    # food_data = food_data["select * from tbl_customer_iot_data tcid where customer_id='261cb253-f11a-461c-9dc0-b608549c4454'"]

with open(r"C:\Users\Vishwas\Downloads\261cb253-f11a-461c-9dc0-b608549c4454_CGM.json") as f:
    cgm_data = json.load(f)
    cgm_data = cgm_data["select COALESCE(ROUND(glucose_reading * 18.08, 2), 0) AS glucose_reading, glucose_uom, cgm_device_id, glucose_warning, reading_date_time AT TIME ZONE 'UTC' AT TIME ZONE 'Asia/Kolkata' AS reading_date_time\r\n                    from tbl_customer_cgm_data\r\n                    where customer_id = '261cb253-f11a-461c-9dc0-b608549c4454'\r\n                    order by reading_date_time desc"]

In [ ]:
from datetime import datetime,timedelta

end_time = datetime.strptime("2025-03-11 10:15", "%Y-%m-%d %H:%M")
start_time = end_time - timedelta(minutes=15)

filtered_meals = []
for meal in food_data:
    # Clean and parse time (remove AM/PM)
    clean_time_str = meal["mealTime"].replace("AM", "").replace("PM", "").strip()
    meal_datetime = datetime.strptime(clean_time_str, "%Y-%m-%d %H:%M")
    # print(meal_datetime,type(meal_datetime))
    if start_time <= meal_datetime < end_time:
        try:
            meal_info = {
                "mealType": meal["mealType"],
                "mealTime": meal["mealTime"],
                "foodName": meal["food"]["foodName"],
                "nutrition": {
                    "protein": str(round(meal["food"]["nutrition"]["macro"]["protein"]["value"],2)) +" "+ str(meal["food"]["nutrition"]["macro"]["protein"]["unit"]),
                    "energy": str(meal["food"]["nutrition"]["macro"]["energy"]["value"]) +" "+ str(meal["food"]["nutrition"]["macro"]["energy"]["unit"]),
                    "total_dietary_fibre": str(meal["food"]["nutrition"]["macro"]["total_dietary_fibre"]["value"])+" "+str(meal["food"]["nutrition"]["macro"]["total_dietary_fibre"]["unit"]),
                    "carbohydrates": str(meal["food"]["nutrition"]["macro"]["carbohydrates"]["value"])+" "+str(meal["food"]["nutrition"]["macro"]["carbohydrates"]["unit"])
                }
            }
        except:
            pass
        
        filtered_meals.append(meal_info)

In [62]:
def fetch_meals(time):
      filtered_meals = []
      time = datetime.strptime(time, "%Y-%m-%dT%H:%M:%S.%fZ")
      end_time = time.strftime("%Y-%m-%d %H:%M")
      
      end_time = datetime.strptime(end_time, "%Y-%m-%d %H:%M")
      start_time = end_time - timedelta(minutes=15)
      # exit
      for meal in food_data:
            clean_time_str = meal["mealTime"].replace("AM", "").replace("PM", "").strip()
            meal_datetime = datetime.strptime(clean_time_str, "%Y-%m-%d %H:%M")
            if start_time <= meal_datetime < end_time:
                  try:
                        f_meal = {
                        "mealType": meal["mealType"],
                        "mealTime": meal["mealTime"],
                        "foodName": meal["food"]["foodName"],
                        "nutrition": {
                              "protein": str(round(meal["food"]["nutrition"]["macro"]["protein"]["value"],2)) +" "+ str(meal["food"]["nutrition"]["macro"]["protein"]["unit"]),
                              "energy": str(meal["food"]["nutrition"]["macro"]["energy"]["value"]) +" "+ str(meal["food"]["nutrition"]["macro"]["energy"]["unit"]),
                              "total_dietary_fibre": str(meal["food"]["nutrition"]["macro"]["total_dietary_fibre"]["value"])+" "+str(meal["food"]["nutrition"]["macro"]["total_dietary_fibre"]["unit"]),
                              "carbohydrates": str(meal["food"]["nutrition"]["macro"]["carbohydrates"]["value"])+" "+str(meal["food"]["nutrition"]["macro"]["carbohydrates"]["unit"])
                        }
                        }
                        filtered_meals.append(f_meal)
                  except:
                        print("passed")
                        pass
      return filtered_meals

In [63]:
fm = fetch_meals("2025-03-10T10:15:00.000Z")

In [64]:
# timestamp = "2025-02-19T10:25:01.709Z"
# datetime_object = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%fZ")
# formatted_time = datetime_object.strftime("%Y-%m-%d %H:%M")
# print(formatted_time)

In [71]:
reference_time_str = "2025-03-11T10:15:00.000Z"
reference_time = datetime.strptime(reference_time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
start_time = reference_time - timedelta(hours=2)

# Filter readings within the last 2 hours before the reference time
filtered_readings = [
    {
        "glucose_reading": str(reading["glucose_reading"]) + " "+"mg/dL",
        "reading_date_time": reading["reading_date_time"],
        "meals_data": fetch_meals(reading["reading_date_time"])
    }
    for reading in cgm_data
    if start_time <= datetime.strptime(reading["reading_date_time"], "%Y-%m-%dT%H:%M:%S.%fZ") <= reference_time
]

# Output the result
for reading in filtered_readings:
    print(reading)

{'glucose_reading': '75.94 mg/dL', 'reading_date_time': '2025-03-11T10:15:00.000Z', 'meals_data': [{'mealType': 'EARLY MORNING', 'mealTime': '2025-03-11 10:00 AM', 'foodName': 'Bhoplya Cha Bharit', 'nutrition': {'protein': '0.84 g', 'energy': '33.93884925842285 kcal', 'total_dietary_fibre': '2.559999942779541 g', 'carbohydrates': '4 g'}}, {'mealType': 'EARLY MORNING', 'mealTime': '2025-03-11 10:00 AM', 'foodName': 'Bhoplya Cha Bharit', 'nutrition': {'protein': '0.84 g', 'energy': '33.93884925842285 kcal', 'total_dietary_fibre': '2.559999942779541 g', 'carbohydrates': '4 g'}}, {'mealType': 'EARLY MORNING', 'mealTime': '2025-03-11 10:00 AM', 'foodName': 'Bhoplya Cha Bharit', 'nutrition': {'protein': '0.84 g', 'energy': '33.93884925842285 kcal', 'total_dietary_fibre': '2.559999942779541 g', 'carbohydrates': '4 g'}}, {'mealType': 'EARLY MORNING', 'mealTime': '2025-03-11 10:00 AM', 'foodName': 'Bhoplya Cha Bharit', 'nutrition': {'protein': '0.84 g', 'energy': '33.93884925842285 kcal', 'tota

In [1]:
import json
with open(r"C:\Users\Vishwas\Documents\food_fabricated.json") as f:
    food_data2 = json.load(f)
    # food_data = food_data["select * from tbl_customer_iot_data tcid where customer_id='261cb253-f11a-461c-9dc0-b608549c4454'"]

with open(r"C:\Users\Vishwas\Documents\cgm_fabricated.json") as f:
    cgm_data2 = json.load(f)


In [2]:
from datetime import datetime,timedelta

def fetch_meals(time):
      filtered_meals = []
      # time = datetime.strptime(time, "%Y-%m-%dT%H:%M:%S.%fZ")
      # end_time = time.strftime("%Y-%m-%d %H:%M")
      
      end_time = datetime.strptime(time, "%d-%m-%Y %H:%M")
      start_time = end_time - timedelta(minutes=15)
      # exit
      for meal in food_data2:
            # clean_time_str = meal["mealTime"].replace("AM", "").replace("PM", "").strip()
            meal_datetime = datetime.strptime(meal['mealTime'], "%d-%m-%Y %H:%M")
            if start_time < meal_datetime <= end_time:
                  try:
                        # f_meal = {
                        # "mealType": meal["mealType"],
                        # "mealTime": meal["mealTime"],
                        # "foodName": meal["food"]["foodName"],
                        # "nutrition": {
                        #       "protein": str(round(meal["food"]["nutrition"]["macro"]["protein"]["value"],2)) +" "+ str(meal["food"]["nutrition"]["macro"]["protein"]["unit"]),
                        #       "energy": str(meal["food"]["nutrition"]["macro"]["energy"]["value"]) +" "+ str(meal["food"]["nutrition"]["macro"]["energy"]["unit"]),
                        #       "total_dietary_fibre": str(meal["food"]["nutrition"]["macro"]["total_dietary_fibre"]["value"])+" "+str(meal["food"]["nutrition"]["macro"]["total_dietary_fibre"]["unit"]),
                        #       "carbohydrates": str(meal["food"]["nutrition"]["macro"]["carbohydrates"]["value"])+" "+str(meal["food"]["nutrition"]["macro"]["carbohydrates"]["unit"])
                        # }
                        # }
                        # print("true")
                        filtered_meals.append(meal)
                  except:
                        print("passed")
                        pass
      return filtered_meals

In [3]:
import pandas as pd
def fetch_steps(time):
      try:
            time = datetime.strptime(time, "%d-%m-%Y %H:%M")
            # print(time)
            df = pd.read_csv(r"c:\Users\Vishwas\Documents\fabricated_step.csv")
            for index, row in df.iterrows():
                  
                  ctime = datetime.strptime(row['Date Time'], "%m-%d-%Y %H:%M")
                  # print(ctime)
                  if ctime == time:
                        return str(row['Steps'])+" steps"
            return 0
      except:
            return 0


In [9]:
reference_time_str = "01-09-2024 18:00"
reference_time = datetime.strptime(reference_time_str, "%d-%m-%Y %H:%M")
start_time = reference_time - timedelta(hours=8)

# Filter readings within the last 2 hours before the reference time
filtered_readings = [
    {
        "glucose_reading": str(reading["glucose_reading"]) + " "+"mg/dL",
        "reading_date_time": reading["reading_date_time"],
        "meals_data": fetch_meals(reading["reading_date_time"]),
        "Activity": fetch_steps(reading['reading_date_time'])
    }
    for reading in cgm_data2
    if start_time <= datetime.strptime(reading["reading_date_time"], "%d-%m-%Y %H:%M") <= reference_time
]

# Output the result
for reading in filtered_readings:
    print(reading)

{'glucose_reading': '121.0 mg/dL', 'reading_date_time': '01-09-2024 10:00', 'meals_data': [], 'Activity': '5.0 steps'}
{'glucose_reading': '135.7 mg/dL', 'reading_date_time': '01-09-2024 10:15', 'meals_data': [], 'Activity': 0}
{'glucose_reading': '135.7 mg/dL', 'reading_date_time': '01-09-2024 10:30', 'meals_data': [{'mealType': 'Snack', 'mealTime': '01-09-2024 10:30', 'foodName': 'Tea', 'nutrition': {'Protein': '3.58 g', 'Energy': '101.45 Kcal', 'Carbohydrate': '12.64 g', 'Fiber': '0.0 g'}}, {'mealType': 'Snack', 'mealTime': '01-09-2024 10:30', 'foodName': 'Marie light biscuit', 'nutrition': {'Protein': '0.94 g', 'Energy': '45.7 Kcal', 'Carbohydrate': '7.4 g', 'Fiber': '0.0 g'}}], 'Activity': 0}
{'glucose_reading': '169.2 mg/dL', 'reading_date_time': '01-09-2024 10:45', 'meals_data': [], 'Activity': 0}
{'glucose_reading': '196.0 mg/dL', 'reading_date_time': '01-09-2024 11:00', 'meals_data': [], 'Activity': '50.0 steps'}
{'glucose_reading': '198.9 mg/dL', 'reading_date_time': '01-09-2

In [10]:
# prompt = f"Here is a timeline of glucose readings and associated meals for the date and time {entry['reading_date_time']}"
prompt = ""
for i, entry in enumerate(filtered_readings, 1):
    prompt += f"{i}. Time: {entry['reading_date_time']}\n"
    prompt += f"   Glucose: {entry['glucose_reading']}\n"
    
    if entry["meals_data"]:
        prompt += f"   Food Intake:\n"
        for meal in entry["meals_data"]:
            nut = meal["nutrition"]
            prompt += (
                f"   - {meal['foodName']} (Protein: {nut['Protein']}, "
                f"Carbs: {nut['Carbohydrate']}, Energy: {nut['Energy']}, Fiber: {nut['Fiber']})\n"
            )
    if not entry['meals_data']:
        prompt += "   Food Intake: None\n"
    if entry['Activity']:
        prompt += f"   Activity:  {entry['Activity']}\n"
    if not entry['Activity']:
        prompt += f"   Activity: None\n" 
        
prompt += "\n Based on all the above observations help me suggest the reason for the glucose levels at "+entry['reading_date_time']

In [11]:
print(prompt)

1. Time: 01-09-2024 10:00
   Glucose: 121.0 mg/dL
   Food Intake: None
   Activity:  5.0 steps
2. Time: 01-09-2024 10:15
   Glucose: 135.7 mg/dL
   Food Intake: None
   Activity: None
3. Time: 01-09-2024 10:30
   Glucose: 135.7 mg/dL
   Food Intake:
   - Tea (Protein: 3.58 g, Carbs: 12.64 g, Energy: 101.45 Kcal, Fiber: 0.0 g)
   - Marie light biscuit (Protein: 0.94 g, Carbs: 7.4 g, Energy: 45.7 Kcal, Fiber: 0.0 g)
   Activity: None
4. Time: 01-09-2024 10:45
   Glucose: 169.2 mg/dL
   Food Intake: None
   Activity: None
5. Time: 01-09-2024 11:00
   Glucose: 196.0 mg/dL
   Food Intake: None
   Activity:  50.0 steps
6. Time: 01-09-2024 11:15
   Glucose: 198.9 mg/dL
   Food Intake: None
   Activity: None
7. Time: 01-09-2024 11:30
   Glucose: 187.2 mg/dL
   Food Intake: None
   Activity: None
8. Time: 01-09-2024 11:45
   Glucose: 168.2 mg/dL
   Food Intake: None
   Activity: None
9. Time: 01-09-2024 12:00
   Glucose: 162.6 mg/dL
   Food Intake: None
   Activity:  55.0 steps
10. Time: 01-09-

In [12]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

client = OpenAI( api_key=os.environ.get("OPENAI_API_KEY"))

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system",
         "content":
         """
              You are a smart glucose monitoring assistant. Analyze continuous glucose monitor (CGM) data recorded every 15 minutes along with corresponding meal intake, sleep duration, and activity levels (steps walked). Review the entire timeline of data, but generate a user-friendly notification only if:
                - Glucose > 180 mg/dL → classify as a "spike" and identify the most probable cause
                - Glucose < 80 mg/dL → classify as a "drop" and identify the most probable cause
                - Any other value (80-180 mg/dL) → classify as "within normal range"
              Evaluate contributing factors when generating glucose alerts. Consider the following:
                - Meals: Check for recent food intake, especially high glycemic foods that may cause spikes.
                - Skipped meals or fasting: May lead to drops or instability in glucose.
                - Poor sleep (<6 hours): Especially in the morning (before 10:00 AM), may explain elevated glucose or fatigue.
                - Morning effects (dawn phenomenon): In early hours, natural hormone release (cortisol, growth hormone) may elevate glucose before breakfast.
                - Activity: Low or no post-meal movement may contribute to postprandial spikes due to reduced glucose uptake by muscles.
              Generate a concise alert (max 30 words) that includes the timestamp, glucose value, the most probable reason, and a brief suggestion to improve. Ensure the message is clear and mobile-friendly.

            Example Notification Output:
            1.   Glucose spike (182 mg/dL) at 06:45, likely due to early morning hormone changes. Try avoiding late-night snacks and see how your morning readings go.
            2.   Glucose spike (192 mg/dL), due to white rice at 13:00. Consider a light walk post-meal.
            3.   Glucose drop (72 mg/dL) after skipping lunch. Your energy dipped. Try not to delay meals—have a small snack like a banana or a handful of roasted chana.
            4.   Glucose (105 mg/dL) at 08:15. Slightly elevated, possibly due to stress from poor sleep. A brisk 10-minute walk can help balance energy and improve glucose control.
            5.   Looking great and healthy (92 mg/dL) at 10:00. Balanced meal + solid sleep (7.5h) supports stability.

            **note: please make sure the response must be as one of the example notification and be particular while using the terms spike and drop.**
                
        """
         },
        {
            "role": "user",
            "content": prompt
            
        },
    ],
)

print(completion.choices[0].message.content)

Glucose drop (68.5 mg/dL) at 18:00, likely after earlier high readings and limited food intake. Consider a balanced meal earlier in the day to maintain steady glucose levels.


o4-mini: Glucose spike (196 mg/dL) at 11:00, likely due to tea and biscuits at 10:30. Consider a short post-snack walk to help lower your levels.

o3: Glucose spike (196 mg/dL) at 11:00, likely from sweet tea and biscuits with little post‑snack movement. Try swapping to unsweetened tea and take a brisk 10‑minute walk.

4o-mini: Glucose spike (196 mg/dL) at 11:00, likely due to tea and biscuits at 10:30 with no post-snack activity. Try a brisk 10‑minute walk to improve uptake.

o3-mini: Glucose spike (196 mg/dL) at 11:00, likely due to the tea and biscuit intake at 10:30. A brisk walk post-snack may help manage your levels.

4o: Glucose spike (196 mg/dL) at 11:00 due to persistent morning elevation. Early hormone release likely cause. Consider monitoring exercise and meal patterns to reduce morning spikes.


In [34]:
print(prompt)

1. Time: 01-09-2024 06:30
   Glucose: 114.0 mg/dL
   Meals: None
2. Time: 01-09-2024 06:45
   Glucose: 117.0 mg/dL
   Meals: None
3. Time: 01-09-2024 07:00
   Glucose: 127.0 mg/dL
   Meals: None
4. Time: 01-09-2024 07:15
   Glucose: 195.0 mg/dL
   Meals: None
5. Time: 01-09-2024 07:30
   Glucose: 239.0 mg/dL
   Meals: None
6. Time: 01-09-2024 07:45
   Glucose: 245.0 mg/dL
   Meals: None
7. Time: 01-09-2024 08:00
   Glucose: 211.0 mg/dL
   Meals: None
8. Time: 01-09-2024 08:15
   Glucose: 195.0 mg/dL
   Meals: None
9. Time: 01-09-2024 08:30
   Glucose: 174.0 mg/dL
   Meals:
   - Idli  (Protein: 4.35 g, Carbs: 29.05 g, Energy: 143.01 Kcal, Fiber: 1.89 g)
   - Vada (Protein: 4.84 g, Carbs: 10.7 g, Energy: 175.03 Kcal, Fiber: 2.5 g)
   - Sambar (Protein: 5.18 g, Carbs: 19.7 g, Energy: 140.82 Kcal, Fiber: 4.13 g)
10. Time: 01-09-2024 08:45
   Glucose: 158.0 mg/dL
   Meals: None
11. Time: 01-09-2024 09:00
   Glucose: 88.2 mg/dL
   Meals: None
12. Time: 01-09-2024 09:15
   Glucose: 89.1 mg/dL

In [151]:
print(prompt)

1. Time: 01-09-2024 05:30
   Glucose: 84.0 mg/dL
   Meals: None

2. Time: 01-09-2024 05:45
   Glucose: 92.0 mg/dL
   Meals: None

3. Time: 01-09-2024 06:00
   Glucose: 98.0 mg/dL
   Meals: None

4. Time: 01-09-2024 06:15
   Glucose: 106.0 mg/dL
   Meals: None

5. Time: 01-09-2024 06:30
   Glucose: 114.0 mg/dL
   Meals: None

6. Time: 01-09-2024 06:45
   Glucose: 117.0 mg/dL
   Meals: None

7. Time: 01-09-2024 07:00
   Glucose: 127.0 mg/dL
   Meals: None

8. Time: 01-09-2024 07:15
   Glucose: 195.0 mg/dL
   Meals: None

9. Time: 01-09-2024 07:30
   Glucose: 239.0 mg/dL
   Meals: None

10. Time: 01-09-2024 07:45
   Glucose: 245.0 mg/dL
   Meals: None

11. Time: 01-09-2024 08:00
   Glucose: 211.0 mg/dL
   Meals: None

12. Time: 01-09-2024 08:15
   Glucose: 195.0 mg/dL
   Meals: None

13. Time: 01-09-2024 08:30
   Glucose: 174.0 mg/dL
   Meals: None

14. Time: 01-09-2024 08:45
   Glucose: 158.0 mg/dL
   Meals:
   - Idli  (Protein: 4.35 g, Carbs: 29.05 g, Energy: 143.01 Kcal, Fiber: 1.89 g)

In [ ]:
reference_time_str = "01-09-2024 10:30"
reference_time = datetime.strptime(reference_time_str, "%d-%m-%Y %H:%M")
start_time = reference_time - timedelta(hours=2)
print(reference_time)
print(start_time)

2024-01-09 10:30:00
2024-01-09 08:30:00


In [97]:
for cgm in cgm_data2:
      print(start_time)
      print(datetime.strptime(cgm["reading_date_time"], "%d-%m-%Y %H:%M"))
      print(reference_time)
      if start_time <= datetime.strptime(cgm["reading_date_time"], "%d-%m-%Y %H:%M") <= reference_time:
            print(start_time)
            print(datetime.strptime(cgm["reading_date_time"], "%d-%m-%Y %H:%M"))
            print(reference_time)

2024-01-09 08:30:00
2024-09-01 00:00:00
2024-01-09 10:30:00
2024-01-09 08:30:00
2024-09-01 00:15:00
2024-01-09 10:30:00
2024-01-09 08:30:00
2024-09-01 00:30:00
2024-01-09 10:30:00
2024-01-09 08:30:00
2024-09-01 00:45:00
2024-01-09 10:30:00
2024-01-09 08:30:00
2024-09-01 01:00:00
2024-01-09 10:30:00
2024-01-09 08:30:00
2024-09-01 01:15:00
2024-01-09 10:30:00
2024-01-09 08:30:00
2024-09-01 01:30:00
2024-01-09 10:30:00
2024-01-09 08:30:00
2024-09-01 01:45:00
2024-01-09 10:30:00
2024-01-09 08:30:00
2024-09-01 02:00:00
2024-01-09 10:30:00
2024-01-09 08:30:00
2024-09-01 02:15:00
2024-01-09 10:30:00
2024-01-09 08:30:00
2024-09-01 02:30:00
2024-01-09 10:30:00
2024-01-09 08:30:00
2024-09-01 02:45:00
2024-01-09 10:30:00
2024-01-09 08:30:00
2024-09-01 03:00:00
2024-01-09 10:30:00
2024-01-09 08:30:00
2024-09-01 03:15:00
2024-01-09 10:30:00
2024-01-09 08:30:00
2024-09-01 03:30:00
2024-01-09 10:30:00
2024-01-09 08:30:00
2024-09-01 03:45:00
2024-01-09 10:30:00
2024-01-09 08:30:00
2024-09-01 04:00:00
